In [ ]:
from src.utils.config_loader import load_yaml_config
from src.tuning.param_grid import generate_param_grid
from pprint import pprint as pp

config = load_yaml_config("../../src/tuning/configs/modset1.yaml")
param_grid = generate_param_grid("ARIMA", config)
pp(param_grid)

In [ ]:
# For testing only: this can be used to simplify the dictionary returned by `get_params()`
# so that the param grid has only one value for each parameter
def simplify_dict(d):
    for key, value in d.items():
        if isinstance(value, list):
            d[key] = value[:1]  # Keep only the first element of the list (max length 1)
        #elif isinstance(value, int):
            #d[key] = value[:1]  # Keep only the first element of the list (max length 1)
    return d

In [ ]:
from sktime.forecasting.model_selection import ForecastingGridSearchCV
from sktime.split import ExpandingSlidingWindowSplitter
from sktime.performance_metrics.forecasting import MeanSquaredError
from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA

y = load_airline()
fh = [1, 2, 3, 4, 5, 6]
cv = ExpandingSlidingWindowSplitter(
    fh=fh, initial_window=12, step_length=12, max_expanding_window_length=24 * 12
)
forecaster = ARIMA()
gscv = ForecastingGridSearchCV(
    forecaster=forecaster,
    param_grid=simplify_dict(param_grid), # Simplify the dictionary so only first values are tested
    cv=cv,
    scoring=MeanSquaredError(square_root=True),
)
gscv.fit(y)
y_pred = gscv.predict(fh)